# importlib - Python’s Import Mechanism
Purpose:	The importlib module exposes the implementation of Python’s import statement.

The importlib module includes functions that implement Python’s import mechanism for loading code in packages and modules. It is one access point to importing modules dynamically, and useful in some cases where the name of the module that needs to be imported is unknown when the code is written (for example, for plugins or extensions to an application).

## Example Package

The examples in this section use a package called example with __init__.py.

In [1]:
# example/__init__.py
print('Importing example package')

Importing example package


In [2]:
# example/submodule.py
print('Importing submodule')

Importing submodule


## Module Types

Python supports several styles of modules. Each requires its own handling when opening the module and adding it to the namespace, and support for the formats varies by platform. 

For example, under Microsoft Windows, shared libraries are loaded from files with extensions .dll or .pyd, instead of .so. The extensions for C modules may also change when using a debug build of the interpreter instead of a normal release build, since they can be compiled with debug information included as well. If a C extension library or other module is not loading as expected, use the constants defined in importlib.machinery to find the supported types for the current platform, and the parameters for loading them.

In [3]:
# importlib_suffixes.py
import importlib.machinery

SUFFIXES = [
    ('Source:', importlib.machinery.SOURCE_SUFFIXES),
    ('Debug:',
     importlib.machinery.DEBUG_BYTECODE_SUFFIXES),
    ('Optimized:',
     importlib.machinery.OPTIMIZED_BYTECODE_SUFFIXES),
    ('Bytecode:', importlib.machinery.BYTECODE_SUFFIXES),
    ('Extension:', importlib.machinery.EXTENSION_SUFFIXES),
]


def main():
    tmpl = '{:<10}  {}'
    for name, value in SUFFIXES:
        print(tmpl.format(name, value))

if __name__ == '__main__':
    main()

Source:     ['.py']
Debug:      ['.pyc']
Optimized:  ['.pyc']
Bytecode:   ['.pyc']
Extension:  ['.cpython-35m-darwin.so', '.abi3.so', '.so']


The return value is a sequence of tuples containing the file extension, mode to use for opening the file containing the module, and a type code from a constant defined in the module. This table is incomplete, because some of the importable module or package types do not correspond to single files.

## Importing Modules

The high level API in importlib makes it simple to import a module given an absolute or relative name. When using a relative module name, specify the package containing the module as a separate argument.

In [4]:
#importlib_import_module.py
import importlib

print('importing m1')
m1 = importlib.import_module('example.submodule')
print('m1 - {}'.format(m1))

print('importing m2')
m2 = importlib.import_module('.submodule', package='example')
print('m2 - {}'.format(m2))

print(m1 is m2)

importing m1
Importing example package
Importing submodule
m1 - <module 'example.submodule' from '/Users/binyang/GitHub/Py3MOTW/Module_and_Package/example/submodule.py'>
importing m2
m2 - <module 'example.submodule' from '/Users/binyang/GitHub/Py3MOTW/Module_and_Package/example/submodule.py'>
True


If the module cannot be imported, import_module() raises ImportError.

In [5]:
# importlib_import_module_error.py
import importlib


try:
    importlib.import_module('example.nosuchmodule')
except ImportError as err:
    print('Error:', err)

Error: No module named 'example.nosuchmodule'


To reload an existing module, use reload().

In [6]:
# importlib_reload.py
import importlib

m1 = importlib.import_module('example.submodule')
print(m1)

m2 = importlib.reload(m1)
print(m1 is m2)

<module 'example.submodule' from '/Users/binyang/GitHub/Py3MOTW/Module_and_Package/example/submodule.py'>
Importing submodule
True


The return value from reload() is the new module. Depending on which type of loader was used, it may be the same module instance.

## Loaders

The lower-level API in importlib provides access to the loader objects, as described in Modules and Imports from the section on the sys module. To get a loader for a module, use find_loader(). Then to retrieve the module, use the loader’s load_module() method.

In [7]:
# importlib_find_loader.py
import importlib

# deprecated importlib.find_loader
# loader = importlib.find_loader('example')
# print('Loader:', loader)

# recommended importlib.util.find_spec()
loader_spec = importlib.util.find_spec('example')
print('\nLoader spec: \n', loader_spec)

# recommended: to replace find_loader
loader_new = importlib.util.find_spec('example').loader
print('\nLoader:', loader_new)

print()
m = loader_new.load_module()
print('\nModule:', m)


Loader spec: 
 ModuleSpec(name='example', loader=<_frozen_importlib_external.SourceFileLoader object at 0x1046c0470>, origin='/Users/binyang/GitHub/Py3MOTW/Module_and_Package/example/__init__.py', submodule_search_locations=['/Users/binyang/GitHub/Py3MOTW/Module_and_Package/example'])

Loader: <_frozen_importlib_external.SourceFileLoader object at 0x1046c0470>

Importing example package

Module: <module 'example' from '/Users/binyang/GitHub/Py3MOTW/Module_and_Package/example/__init__.py'>


Submodules within packages need to be loaded separately using the path from the package. In the following example, the package is loaded first and then its path is passed to find_loader() to create a loader capable of loading the submodule.

In [8]:
# importlib_submodule.py
import importlib

# deprecating
# pkg_loader = importlib.find_loader('example')

pkg_loader = importlib.util.find_spec('example').loader
pkg = pkg_loader.load_module()

# loader = importlib.util.find_spec('submodule', pkg.__path__).loader
loader = importlib.find_loader('submodule', pkg.__path__)
print('Loader:', loader)
print('pkg.__path__:', pkg.__path__)

m = loader.load_module()
print('Module:', m)

Importing example package
Loader: <_frozen_importlib_external.SourceFileLoader object at 0x1046c0c50>
pkg.__path__: ['/Users/binyang/GitHub/Py3MOTW/Module_and_Package/example']
Importing submodule
Module: <module 'submodule' from '/Users/binyang/GitHub/Py3MOTW/Module_and_Package/example/submodule.py'>


/Users/binyang/anaconda/envs/py3.5/lib/python3.5/site-packages/ipykernel/__main__.py:11: DeprecationWarning: Use importlib.util.find_spec() instead.


Unlike with import_module(), the name of the submodule should be given without any relative path prefix, since the loader will already be constrained by the package’s path.